Tutorial link : https://www.tensorflow.org/tutorials/keras/text_classification

In [1]:
!pip install tensorflow
!pip install numpy

  Using cached tensorflow-2.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.

In [2]:
import tensorflow as tf
import numpy as np
import os

2024-07-11 12:32:11.506378: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 12:32:11.507298: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:32:11.509096: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:32:11.514977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 12:32:11.526950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

# Download Dataset

In [11]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(
    "aclImdb_v1",
    url,
    untar = True,
    cache_dir = ".",
    cache_subdir = "."
)

dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")
dataset_dir

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 148s 2us/step


'././aclImdb'

In [15]:
os.listdir(dataset_dir)

['imdbEr.txt', 'README', 'test', 'train', 'imdb.vocab']

# Fetch dataset from download directory

In [16]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(          
    'aclImdb/train',
    batch_size = batch_size,        
    validation_split = 0.2,
    subset = "training",                        #this part of /train data will be used for training
    seed = seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [25]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',                      #this part of /train data will be used for validation
    seed = seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [26]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size             #simply load the data from /test
)

Found 25000 files belonging to 2 classes.


In [35]:
for i , j in raw_train_ds.take(1):
    print(j)

# here raw_train_ds contains multiple batches of size 32
# each batch contains 2 tensors
# i is the tensor of 32 strings, and j is a tensor containing 32 labels
# a tensor abc, can be converted into a numpy array using abc.numpy()

tf.Tensor([1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0], shape=(32,), dtype=int32)


2024-07-11 14:40:56.793634: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review" , text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Review b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />There\'s a 

2024-07-11 13:40:06.353481: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Prepare dataset for training

1) We'll remove the html tags present in the text strings.

In [39]:
import re
import string

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")                          #replace br tags with space
    return tf.strings.regex_replace(stripped_html, "[%s]" % re.escape(string.punctuation), "")  #remove punctuations

2) Now create a vectorisation layer that'll standardise, tokenise, vectorise our data

In [40]:
max_features = 10000
sequence_length = 250

vectorize_layer = tf.keras.layers.TextVectorization(                #A preprocessing layer which maps text features to integer sequences.
    standardize = custom_standardization,
    max_tokens = max_features,                      #Maximum size of the vocabulary for this layer
    output_mode = "int",
    output_sequence_length = sequence_length        #results in exact THIS length outputs
)

#read this https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

3) Since we need to vectorize, we need to pass only-text dataset. ie remove labels

In [41]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

2024-07-11 15:06:25.994330: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Build model

In [42]:
embedding_dims = 16

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(max_features, embedding_dims),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = "sigmoid")                #sigmoid, because binary classifier, no of class = 2
])

In [44]:
model.compile(
    optimizer = "adam",
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

In [45]:
model.fit(raw_train_ds, validation_data = raw_val_ds, epochs = 10)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5827 - loss: 0.6814 - val_accuracy: 0.7186 - val_loss: 0.6136
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7545 - loss: 0.5813 - val_accuracy: 0.8114 - val_loss: 0.5006
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8211 - loss: 0.4710 - val_accuracy: 0.8396 - val_loss: 0.4246
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8479 - loss: 0.4004 - val_accuracy: 0.8504 - val_loss: 0.3820
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8631 - loss: 0.3535 - val_accuracy: 0.8644 - val_loss: 0.3506
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8782 - loss: 0.3220 - val_accuracy: 0.8672 - val_loss: 0.3312
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8906 - loss: 0.2943 - val_accuracy: 0.8608 - val_loss: 0.3277
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8948 - loss: 0.2766 - val_accuracy: 

In [46]:
model.evaluate(raw_test_ds, verbose = 2)

782/782 - 4s - 4ms/step - accuracy: 0.9064 - loss: 0.2453


[0.24532447755336761, 0.9064000248908997]

# Predicting for solitary data

In [52]:
examples = tf.constant([                #Creates a constant tensor from a tensor-like object.
    "i liked it",
    "That was horrible"
])

model.predict(examples)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([[0.40422457],
       [0.19741124]], dtype=float32)